# بارگذاری کتابخانه ها

In [ ]:
import numpy as np                    
import pandas as pd                   # پردازش داده
import matplotlib.pyplot as plt       # نمایش نمودارها
import seaborn as sns                 # نمایش نمودارها
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix 
import warnings                       # مخفی کردن پیام های هشدار
warnings.filterwarnings("ignore")
%matplotlib inline                    

# بارگذاری داده ها 

In [ ]:
# خواندن فایل مجموعه داده ها 
df = pd.read_csv("../input/advertising.csv") 

# بررسی داده ها

In [ ]:
df.head(10) # بررسی ده سطر اول فایل داده ایمپورت شده

# نوع داده و طول متغیرها

In [ ]:
df.info() # جزئیاتی در مورد متغیر ها را دریافت می کنیم

# بررسی داده های تکراری

In [ ]:
df.duplicated().sum() # رکوردهای تکراری را نشان میدهد

# شناسایی متغیرهای عددی و رسته ای

In [ ]:
df.columns # نمایش عنوان ستون ها

In [ ]:
df.select_dtypes(include = ['object']).columns # متغیرهای رسته ای که توسط پایتون شناسایی شده اند را نمایش میدهد

In [ ]:
# تخصیص ستون های زیر بعنوان متغیرهای عددی
numeric_cols = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage' ]

In [ ]:
# تخصیص ستون های زیر بعنوان متغیرهای رسته ای
Categorical_cols = [ 'Ad Topic Line', 'City', 'Male', 'Country', 'Clicked on Ad' ]

# خلاصه کردن متغیرهای عددی

In [ ]:
df[numeric_cols].describe()
# از متد بالا برای مشاهده ویژگی های آماری ستون های عددی استفاده می کنیم

نزدیکی زیاد میانگین و میانه به هم بر این دلالت دارد که داده دارای تغییرات غیرنرمال نیست و نیازی به تبدل داده نداریم. که توسط نمودار نیز به خوبی قابل درک است

# خلاصه کردن متغرهای رسته ای

In [ ]:
df[Categorical_cols].describe(include = ['O'])
# از متد بالا برای مشاهده ویژگی های آماری ستون های رسته ای استفاده می کنیم 

از آنجا که شهرهای مختلف (یکتا) داریم و همچنین تعداد زیادی از افراد به در یک شهر نیستند (فرکانس) پس احتمال زیاد این مشخصه قدرت پیش بینی چندانی برای ما به ارمغان نمی آورد. البته مشخصه کشور که تقریبا مشابه این مشخصه است وجود دارد و با توجه به گوناگونی کمتر آن میتوانیم از آن بهره بببریم

### متغیر کشورها

In [ ]:
pd.crosstab(df['Country'], df['Clicked on Ad']).sort_values(1,0, ascending = False).head(10)

In [ ]:
pd.crosstab(index=df['Country'],columns='count').sort_values(['count'], ascending=False).head(10)

به نظر میرسد بیشترین کاربران از دو کشور فرانسه و جمهوری چک و هر کدام به تعداد 9 نفر هستند

# بررسی مقادیر تهی

In [ ]:
df.isnull().sum() # تعداد رکوردهای تهی در هر ستون

# استخراج متغیرهای زمان-تاریخ

مشخصه برچسب زمانی کمک خوبی به ما برای فهم الگوی کاربر در کلیک روی تبلیغات می کند

In [ ]:
# استخراج زمان-تاریخ با استفاده از ستون برچسب زمانی
df['Timestamp'] = pd.to_datetime(df['Timestamp']) 
# تبدیل ستون برچسب زمانی به شی زمان-تاریخ برای ایجاد مشخصه های جدید
df['Month'] = df['Timestamp'].dt.month 
# ایجاد ستون جدید بنام ماه
df['Day'] = df['Timestamp'].dt.day     
# ایجاد ستون جدید بنام روز
df['Hour'] = df['Timestamp'].dt.hour   
# ایجاد ستون جدید بنام ساعت
df["Weekday"] = df['Timestamp'].dt.dayofweek 
# ایجاد ستون جدید بنام روزهای هفته بصورتیکه یکشننه دارای مقدار 6 و دوشنبه دارای مقدار 0 باشد
df = df.drop(['Timestamp'], axis=1) # حدف ستون برچسب زمانی

In [ ]:
df.head() # بررسی و اطمینان از افزوده شدن متغیرهای جدید به مجموعه داده ما

# رسم نمودار متغیر هدف 

In [ ]:
plt.figure(figsize = (14, 6)) 
plt.subplot(1,2,1)            
sns.countplot(x = 'Clicked on Ad', data = df)
plt.subplot(1,2,2)
sns.distplot(df["Clicked on Ad"], bins = 20)
plt.show()

از طریق نمودار می توانیم ببینیم که تعداد کاربرانی که روی تبلیغات کلیک کرده اند برابر با کاربرانی است که روی تبلیغات کلیک نکرده اند که برابر 500 است

In [ ]:
# نمودار مشترک زمان صرف شده در سایت و سن
sns.jointplot(x = "Age", y= "Daily Time Spent on Site", data = df) 

میتوان مشاهده کرد که افراد بین 30 و 40 سال روزانه وقت بیشتری بر روی سایت میگدرانند

# توزیع و رابطه بین متغیرها

In [ ]:
sns.pairplot(df, hue = 'Clicked on Ad', vars = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage'], palette = 'husl')

رابطه بین متغیر هدف و سایر متغیر ها را نشان می دهد. میتوان مشاهده کرد که افزادی که وقت کمتری به سایت بوده اند دارای درآمد کمتری هستند و با افزایش سن نمایل به کلیک روی تبلیغات نیز بیشتر می شود

In [ ]:
plots = ['Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage']
for i in plots:
    plt.figure(figsize = (14, 6))
    plt.subplot(1,2,1)
    sns.boxplot(df[i])
    plt.subplot(1,2,2)
    sns.distplot(df[i],bins= 20)    
    plt.title(i)    
    plt.show()

می توانیم به وضوح ببینیم که میانگین استفاده روزانه از اینترنت و زمان گذرانده روی سایت دارای دو پیک است. این نشان می دهد که دو گروه متفاوت در داده ما وجود دارند   

In [ ]:
print('Oldest person :', df['Age'].max(), 'Years')
print('Youngest person :', df['Age'].min(), 'Years')
print('Average :', df['Age'].mean(), 'Years')

# همبستگی بین متغیرها

In [ ]:
fig = plt.figure(figsize = (12,10))
sns.heatmap(df.corr(), cmap='Blues', annot = True) # درجه همبستگی بر اساس نمدار هیت مپ

درک بهتری به ما از رابطه بین هر کدام از متغیرها می دهد. همبستگی مقداری مابین -1 و 1 دارد. هر قدر ان مقدار بیشتر باشد, میزان همبستگی این دو متغیر بیشتر است. ما انتظار داریم که دو مشخصه مقدار انترنت مورد استفاده روزانه و مقدار زمان صرف شده در سایت دارای همبستگی بیشتری با متغیر هدف ما باشند. همچنین هیچکدام از متغیرهای توصیفی با دیکدیگر همبستگی ندارند

# ترسیم نمودار متغیرهای استخراج شده

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,5))
df['Month'][df['Clicked on Ad']==1].value_counts().sort_index().plot(ax=ax[0])
ax[0].set_title('Months Vs Clicks')
ax[0].set_ylabel('Count of Clicks')
pd.crosstab(df["Clicked on Ad"], df["Month"]).T.plot(kind = 'Bar',ax=ax[1])
plt.tight_layout()
plt.show()

نمودار خطی تعداد کلیک ها به ازای هر ماه را نشان می دهد. نمودا رمیله ای تجمیع شده توزیع متغیر هدف در طول 7 ماه را نشان می دهد. به نظر میرسد دومین ماه بهترین برای کلیک روی تبلیغات می باشد

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,5))
pd.crosstab(df["Clicked on Ad"], df["Hour"]).T.plot(style = [], ax = ax[0])
pd.pivot_table(df, index = ['Weekday'], values = ['Clicked on Ad'],aggfunc= np.sum).plot(kind = 'Bar', ax=ax[1]) # 0 - دوشنبه
plt.tight_layout()
plt.show()

نموار خطی نشان می دهد که کاربران اغلب تمالیل به کلیک روی تبلیغات در صبح زود یا آخراهای روز دارند که میتوان اینگونه توجیه کرد  با توجه به شاغل بودن کاربران فقط در این زمان ها وقت آزاد دارند. همپنین از روی نمودار میله ای روز یکشنبه برای کلیک روی تبلیغ مناسب به نظر مییرسد

## کلیک شده و کلیک نشده

In [ ]:
df.groupby('Clicked on Ad')['Clicked on Ad', 'Daily Time Spent on Site', 'Age', 'Area Income', 
                            'Daily Internet Usage'].mean()

توصیف میانگینی از یک کاربر که آیا روی یک تبلیغ کلیک خواهد کرد یا نه

In [ ]:
df.groupby(['Male','Clicked on Ad'])['Clicked on Ad'].count().unstack()

توزیع کیلک ها با توجه به جنسیت. بنظر میرسد زنان بیشتر بر روی تبلیغات کلیک کرده اند

In [ ]:
hdf = pd.pivot_table(df, index = ['Hour'], columns = ['Male'], values = ['Clicked on Ad'], 
                     aggfunc= np.sum).rename(columns = {'Clicked on Ad':'Clicked'})

cm = sns.light_palette("green", as_cmap=True)
hdf.style.background_gradient(cmap=cm)  # تمام یک ها را که بیانگر کلیک شدن هست برای هر ساعت جمع می کنیم

توزیع بر اساس ساعت و جنسیت. در کل زنان تمایل بیشتری نسبت به مردان در کلیک روی تبلیغات دارند

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,5))
sns.set_style('whitegrid')
sns.countplot(x='Male',hue='Clicked on Ad',data=df,palette='bwr', ax = ax[0])
table = pd.crosstab(df['Weekday'],df['Clicked on Ad'])
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True, ax=ax[1], grid = False) # 0 - دوشنبه
ax[1].set_title('Stacked Bar Chart of Weekday vs Clicked')
ax[1].set_ylabel('Proportion by Day')
ax[1].set_xlabel('Weekday')
plt.tight_layout()
plt.show()

از نمودار پشته ای واضح است که احتمال بیشتری دارد تا یک کاربر روی تبلیغ کلیک کند اگر پنجشنبه باشد

In [ ]:
sns.factorplot(x="Weekday", y="Age", col="Clicked on Ad", data=df, kind="box",size=5, aspect=2.0) 

مقایسه کاربرانی که بر روی تبلیغ کلیک کرده یا نکرده اند از منظر سن و روزهای هفته. واضح است هر قدر سن بالاتر باشد, تمایل به کلیک روی تبلیغات بیشتر است

In [ ]:
sns.factorplot('Month', 'Clicked on Ad', hue='Male', data = df)
plt.show()

# تشخیص داده های پرت مستعد با استفاده از IQR

In [ ]:
for i in numeric_cols:
    stat = df[i].describe()
    print(stat)
    IQR = stat['75%'] - stat['25%']
    upper = stat['75%'] + 1.5 * IQR
    lower = stat['25%'] - 1.5 * IQR
    print('The upper and lower bounds for suspected outliers are {} and {}.'.format(upper, lower))

# ساخت یک مدل ساده بر پایه داده های واقعی

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# تخصیص ستون های عددی به متغیر های ایکس و وای چون مدل صرفا قادر به کار روی اعداد است
X = df[['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Male']]
y = df['Clicked on Ad']

In [ ]:
# جداسازی داده ها به منظور یادگیری و تست
#تست سایز درصد داده هایی است که می خواهیم تخصیص دهیم
#رندم استیت هم اطمینان حاصل کردن از ان است که ک مجموعه خاص از مقادیر تصادفی جدا شود
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# ساخت یک مدل ساده

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
model = logreg.fit(X_train,y_train)
model

# پیش بین ها

In [ ]:
# متد پردیکت ایکس تست ها را درافت می کند در واقع پیش بینی ها را بر اساس مشخصه ها انجام می دهد
predictions = logreg.predict(X_test)
predictions[0:20]

# معیارهای کارایی

حال نیاز داریم تا ببینیم پیش بینی های ما چقدر با داده های آزمایشی واقعی (y_test) مطابقت دارند

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

## نتایج مدل ساده

تایج ارزیابی بدین صورت است:
ماتریس درهم رختگی:
کاربرانی که پیش بینی میشد روی تبلیغات کلیک کندد و واقعا کلیک کردندن 144 نفر بود. کارابرانی که پیش بینی میشد روی تبلیغات کلیک نکنند و کلیک نکردند نیز 158 نفر بود.
کاربرانی که پیش بینی می شد روی تبلیغات کلیک کندد و کلیک نکردند 6 نفر بود. و کارابرانی که پیش بینی می شد روی تبلیغات کلیک نکنند ولی کلیک کردند نیز 24 نفر بود.
نتیجه با توجه بع مجموعه داده ما چندان بد به نظر نمی رسد.

گزارش دسته بندی:
ذقت و صحت هر کدام 0.91 بدست آمده اند یعنی مقادیر پیش بینی شده 91% دقیق و صحیح هستند. پس با اطمینان 91% میتوانیم کلیک کردن یا نکردن یک کاربر را پیش بینی کنیم.


# مهندسی مشخصه

In [ ]:
new_df = df.copy() # صرفا برای حفط چارچوب اصلی داده ها

In [ ]:
# ایجاد نمودار برای بررسی تاثیر متغرهای زمان-تاریخ بر روی متغیر هدف
pp = sns.pairplot(new_df, hue= 'Clicked on Ad', vars = ['Month', 'Day', 'Hour', 'Weekday'], palette= 'husl')

به نظر نمی رسد که روز و ماه و روزهای هفته و ساعت بر روی متغیر هدف تاثیری داشته باشند

In [ ]:
new_df = pd.concat([new_df, pd.get_dummies(new_df['Month'], prefix='Month')], axis=1) 
new_df = pd.concat([new_df, pd.get_dummies(new_df['Weekday'], prefix='Weekday')], axis=1)

In [ ]:
new_df['Hour_bins'] = pd.cut(new_df['Hour'], bins = [0, 5, 11, 17, 23], 
                        labels = ['Hour_0-5', 'Hour_6-11', 'Hour_12-17', 'Hour_18-23'], include_lowest= True)

In [ ]:
new_df = pd.concat([new_df, pd.get_dummies(new_df['Hour_bins'], prefix='Hour')], axis=1)

In [ ]:
# ستون سن
plt.figure(figsize=(25,10))
sns.barplot(new_df['Age'],df['Clicked on Ad'], ci=None)
plt.xticks(rotation=90)

In [ ]:
# بررسی
limit_1 = 18
limit_2 = 35

x_limit_1 = np.size(df[df['Age'] < limit_1]['Age'].unique())
x_limit_2 = np.size(df[df['Age'] < limit_2]['Age'].unique())

plt.figure(figsize=(15,10))
sns.countplot('Age',hue='Clicked on Ad',data=df)
plt.axvspan(-1, x_limit_1, alpha=0.25, color='green')
plt.axvspan(x_limit_1, x_limit_2, alpha=0.25, color='red')
plt.axvspan(x_limit_2, 50, alpha=0.25, color='yellow')

plt.xticks(rotation=90)

In [ ]:
new_df['Age_bins'] = pd.cut(new_df['Age'], bins=[0, 18, 30, 45, 70], labels=['Young','Adult','Mid', 'Elder'])

In [ ]:
sns.countplot('Age_bins',hue='Clicked on Ad',data= new_df) # بررسی

In [ ]:
new_df = pd.concat([new_df, pd.get_dummies(new_df['Age_bins'], prefix='Age')], axis=1) 

In [ ]:
new_df = pd.concat([new_df, pd.get_dummies(new_df['Country'], prefix='Country')], axis=1)

In [ ]:
# حدف مشخصه های اضافی
new_df.drop(['Country', 'Ad Topic Line', 'City', 'Day', 'Month', 'Weekday', 
             'Hour', 'Hour_bins', 'Age', 'Age_bins'], axis = 1, inplace = True)
new_df.head() # بررسی چارچوب داده نهایی

# ساخت مدل رگرسیون لجستیک

In [ ]:
X = new_df.drop(['Clicked on Ad'],1)
y = new_df['Clicked on Ad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# استانداردسازی مشخصه ها
from  sklearn.preprocessing  import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import  statsmodels.api  as sm
from scipy import stats

X2   = sm.add_constant(X_train_std)
est  = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

همانطور ککه می بینیم مشخصه جنسیت مشارکتی در مدل ندارد پس آن را از مدل حذف می کنیم
مشخصه های مشارکت کننده در مدل به شرح زیر هستند:

- زمان صرف شده روزانه در سایت
- اینرنت مصرفی روزانه
- سن
- کشور
- درامد میانگین

In [ ]:
# اعمال رگرسیون لجستیک برای یادگیری
lr = LogisticRegression(penalty="l2", C= 0.1, random_state=42)
lr.fit(X_train_std, y_train)
# پیش بینی با استفاده از مدل
lr_training_pred = lr.predict(X_train_std)
lr_training_prediction = accuracy_score(y_train, lr_training_pred)

print( "Accuracy of Logistic regression training set:",   round(lr_training_prediction,3))

In [ ]:
#اعتبارسنجی متقابل
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(lr, # مدل
                         X_train_std, # ماتریس مشخصه
                         y_train, # بردار هدف
                         cv=kf, # تکنیک اعتبار سنجی متقابل
                         scoring="accuracy",
                         n_jobs=-1) 
print('10 fold CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.model_selection import cross_val_predict
print('The cross validated score for Logistic Regression Classifier is:',round(scores.mean()*100,2))
y_pred = cross_val_predict(lr,X_train_std,y_train,cv=10)
sns.heatmap(confusion_matrix(y_train,y_pred),annot=True,fmt='3.0f',cmap="winter")
plt.title('Confusion_matrix', y=1.05, size=15)

# مدل جنگل تصادفی

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini', n_estimators=400,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=42,n_jobs=-1)
rf.fit(X_train_std,y_train)
# پیشبینی توسط مدل
rf_training_pred = rf.predict(X_train_std)
rf_training_prediction = accuracy_score(y_train, rf_training_pred)

print("Accuracy of Random Forest training set:",   round(rf_training_prediction,3))

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(rf, # مدل
                         X_train_std, # ماتریس مشخصه
                         y_train, # بردار هدف
                         cv=kf, # تکنیک اعتبار سنجی متقابل
                         scoring="accuracy", 
                         n_jobs=-1) 
print('10 fold CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

In [ ]:
from sklearn.model_selection import cross_val_predict
print('The cross validated score for Random Forest Classifier is:',round(scores.mean()*100,2))
y_pred = cross_val_predict(rf,X_train_std,y_train,cv=10)
sns.heatmap(confusion_matrix(y_train,y_pred),annot=True,fmt='3.0f',cmap="winter")
plt.title('Confusion_matrix', y=1.05, size=15)

# ارزیابی کارایی مدل ها

In [ ]:
print ("\n\n ---Logistic Regression Model---")
lr_auc = roc_auc_score(y_test, lr.predict(X_test_std))

print ("Logistic Regression AUC = %2.2f" % lr_auc)
print(classification_report(y_test, lr.predict(X_test_std)))

print ("\n\n ---Random Forest Model---")
rf_roc_auc = roc_auc_score(y_test, rf.predict(X_test_std))

print ("Random Forest AUC = %2.2f" % rf_roc_auc)
print(classification_report(y_test, rf.predict(X_test_std)))

میتوان مشاهده کرد که مدل جنگل تصادفی دارای کارایی بهتری از رگرسیون لجستیک می باشد

# گراف ROC

In [ ]:
# ایجاد گراف ROC
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, lr.predict_proba(X_test_std)[:,1])
rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test, rf.predict_proba(X_test_std)[:,1])


plt.figure()

# رسم ROC رگرسیون لجستیک
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % lr_auc)

# رسم ROC [k'g jwhntd]
plt.plot(rf_fpr, rf_tpr, label='Random Forest Classifier (area = %0.2f)' % rf_roc_auc)


# Base Rate ROC
plt.plot([0,1], [0,1],label='Base Rate')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Graph')
plt.legend(loc="lower right")
plt.show()

# تعیین مشخصه ها بر اساس اهمیتشان در مدل جنگل تصادفی

In [ ]:
columns = X.columns
train = pd.DataFrame(np.atleast_2d(X_train_std), columns=columns) # تبدیل آرایه به چارچوب داده

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances = feature_importances.reset_index()
feature_importances.head(10)

In [ ]:
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(13, 7))

sns.set_color_codes("pastel")
sns.barplot(x="importance", y='index', data=feature_importances[0:10],
            label="Total", color="b")